$$
\newcommand{\ba}{\boldsymbol{a}}
\newcommand{\bb}{\boldsymbol{b}}
\newcommand{\bc}{\boldsymbol{c}}
\newcommand{\bd}{\boldsymbol{d}}
\newcommand{\be}{\boldsymbol{e}}
\newcommand{\bff}{\boldsymbol{f}}
\newcommand{\bg}{\boldsymbol{g}}
\newcommand{\bh}{\boldsymbol{h}}
\newcommand{\bi}{\boldsymbol{i}}
\newcommand{\bj}{\boldsymbol{j}}
\newcommand{\bk}{\boldsymbol{k}}
\newcommand{\bl}{\boldsymbol{l}}
\newcommand{\bm}{\boldsymbol{m}}
\newcommand{\bn}{\boldsymbol{n}}
\newcommand{\bo}{\boldsymbol{o}}
\newcommand{\bp}{\boldsymbol{p}}
\newcommand{\bq}{\boldsymbol{q}}
\newcommand{\br}{\boldsymbol{r}}
\newcommand{\bs}{\boldsymbol{s}}
\newcommand{\bt}{\boldsymbol{t}}
\newcommand{\bu}{\boldsymbol{u}}
\newcommand{\bv}{\boldsymbol{v}}
\newcommand{\bw}{\boldsymbol{w}}
\newcommand{\bx}{\boldsymbol{x}}
\newcommand{\by}{\boldsymbol{y}}
\newcommand{\bz}{\boldsymbol{z}}
\newcommand{\bA}{\boldsymbol{A}}
\newcommand{\bB}{\boldsymbol{B}}
\newcommand{\bC}{\boldsymbol{C}}
\newcommand{\bD}{\boldsymbol{D}}
\newcommand{\bE}{\boldsymbol{E}}
\newcommand{\bF}{\boldsymbol{F}}
\newcommand{\bG}{\boldsymbol{G}}
\newcommand{\bH}{\boldsymbol{H}}
\newcommand{\bI}{\boldsymbol{I}}
\newcommand{\bJ}{\boldsymbol{J}}
\newcommand{\bK}{\boldsymbol{K}}
\newcommand{\bL}{\boldsymbol{L}}
\newcommand{\bM}{\boldsymbol{M}}
\newcommand{\bN}{\boldsymbol{N}}
\newcommand{\bO}{\boldsymbol{O}}
\newcommand{\bP}{\boldsymbol{P}}
\newcommand{\bQ}{\boldsymbol{Q}}
\newcommand{\bR}{\boldsymbol{R}}
\newcommand{\bS}{\boldsymbol{S}}
\newcommand{\bT}{\boldsymbol{T}}
\newcommand{\bU}{\boldsymbol{U}}
\newcommand{\bV}{\boldsymbol{V}}
\newcommand{\bW}{\boldsymbol{W}}
\newcommand{\bX}{\boldsymbol{X}}
\newcommand{\bY}{\boldsymbol{Y}}
\newcommand{\bZ}{\boldsymbol{Z}}
\newcommand{\balpha}{\boldsymbol{\alpha}}
\newcommand{\bbeta}{\boldsymbol{\beta}}
\newcommand{\bgamma}{\boldsymbol{\gamma}}
\newcommand{\bdelta}{\boldsymbol{\delta}}
\newcommand{\bepsilon}{\boldsymbol{\epsilon}}
\newcommand{\blambda}{\boldsymbol{\lambda}}
\newcommand{\bmu}{\boldsymbol{\mu}}
\newcommand{\bnu}{\boldsymbol{\nu}}
\newcommand{\bphi}{\boldsymbol{\phi}}
\newcommand{\bpi}{\boldsymbol{\pi}}
\newcommand{\bsigma}{\boldsymbol{\sigma}}
\newcommand{\btheta}{\boldsymbol{\theta}}
\newcommand{\bomega}{\boldsymbol{\omega}}
\newcommand{\bxi}{\boldsymbol{\xi}}
\newcommand{\bGamma}{\boldsymbol{\Gamma}}
\newcommand{\bDelta}{\boldsymbol{\Delta}}
\newcommand{\bTheta}{\boldsymbol{\Theta}}
\newcommand{\bLambda}{\boldsymbol{\Lambda}}
\newcommand{\bXi}{\boldsymbol{\Xi}}
\newcommand{\bPi}{\boldsymbol{\Pi}}
\newcommand{\bSigma}{\boldsymbol{\Sigma}}
\newcommand{\bUpsilon}{\boldsymbol{\Upsilon}}
\newcommand{\bPhi}{\boldsymbol{\Phi}}
\newcommand{\bPsi}{\boldsymbol{\Psi}}
\newcommand{\bOmega}{\boldsymbol{\Omega}}
$$

# Nonnegative Matrix Factorization
##### Dr. Hua Zhou, Aug 10, 2016
##### SAMSI Opt Program Summer School

Display system information.

In [10]:
versioninfo()

Julia Version 0.4.6
Commit 2e358ce (2016-06-19 17:16 UTC)
Platform Info:
  System: Darwin (x86_64-apple-darwin13.4.0)
  CPU: Intel(R) Core(TM) i7-3720QM CPU @ 2.60GHz
  WORD_SIZE: 64
  BLAS: libopenblas (USE64BITINT DYNAMIC_ARCH NO_AFFINITY Sandybridge)
  LAPACK: libopenblas64_
  LIBM: libopenlibm
  LLVM: libLLVM-3.3


Recall that NNMF finds a low rank approximation to the data matrix $\bX$ by minimizing
$$
    \|\bX - \bV \bW\|_{\text{F}}^2.
$$
The MM algorithm iterate according to
$$
    \bV \gets \bV \, .\times \, (\bX \bW^T) \, ./ \, (\bV \bW \bW^T)
$$
$$
    \bW \gets \bW \, .\times \, (\bV^T \bX) \, ./ \, (\bV^T \bV \bW)
$$

I assume data files or symbolic links to data files are in the current folder. First load in the face data set and display a couple sample faces. Note each row is a face.

In [2]:
X = readdlm("nnmf-2429-by-361-face.txt", ' ', Float64);

Read in the starting values.

In [3]:
V0full = readdlm("V0.txt", ' ', Float64)
W0full = readdlm("W0.txt", ' ', Float64);

Function `nnmf.jl` implements the multiplicative algorithm.

In [4]:
#
# This function fits rank-r nonnegative matrix factorization (NNMF) of
# a data matrix X with nonnegative entries by a multiplicative algorithm.
#
function nnmf(X::Matrix{Float64}, r::Int;
              maxiter::Int=1000, tolfun::Float64=1e-4,
              V::Matrix{Float64} = rand(size(X, 1), r),
              W::Matrix{Float64} = rand(r, size(X, 2)),
              device::Symbol = :CPU, 
              precision::Symbol = :DP)
  # size of data matrix
  m = size(X, 1)
  n = size(X, 2)
  mn = m * n

  if device == :CPU

    # MM loop on CPU
    Vnum = zeros(V)
    Vden = zeros(V)
    Wnum = zeros(W)
    Wden = zeros(W)
    B = V * W
    XminusB = X - B
    obj = vecnorm(XminusB)^2
    for iter = 1:maxiter

      # multiplicative update of V and W
      # V = V .* (X * W') ./ (B * W')
      BLAS.gemm!('N', 'T', 1.0, X, W, 0.0, Vnum)
      BLAS.gemm!('N', 'T', 1.0, B, W, 0.0, Vden)
      for j = 1:r
        @simd for i = 1:m
          @inbounds V[i, j] *= (Vnum[i, j] / Vden[i, j])
        end
      end
      # B = V * W
      BLAS.gemm!('N', 'N', 1.0, V, W, 0.0, B)
      # W = W .* (V' * X) ./ (V' * B)
      BLAS.gemm!('T', 'N', 1.0, V, X, 0.0, Wnum)
      BLAS.gemm!('T', 'N', 1.0, V, B, 0.0, Wden)
      for j = 1:n
        @simd for i = 1:r
          @inbounds W[i, j] *= (Wnum[i, j] / Wden[i, j])
        end
      end
      # B = V * W
      BLAS.gemm!('N', 'N', 1.0, V, W, 0.0, B)

      # check convergence
      objold = obj
      BLAS.blascopy!(mn, X, 1, XminusB, 1)
      BLAS.axpy!(mn, -1.0, B, 1, XminusB, 1)
      obj = vecnorm(XminusB); obj = obj * obj
      if abs(obj - objold) < tolfun * (abs(obj) + 1.0)
        break
      end
    end

    # output
    return V, W

  elseif device == :GPU
    # MM loop on CPU

    # transfer data X, V, W to GPU
    # and load kernel function
    md = CUDArt.CuModule("vecop.ptx", false)
    if precision == :DP
      d_X = CUDArt.CudaArray(X)
      d_V = CUDArt.CudaArray(V)
      d_W = CUDArt.CudaArray(W)
      vmuldiv = CUDArt.CuFunction(md, "vmuldiv_dp")
    elseif precision == :SP
      d_X = CUDArt.CudaArray(map(Float32, X))
      d_V = CUDArt.CudaArray(map(Float32, V))
      d_W = CUDArt.CudaArray(map(Float32, W))
      vmuldiv = CUDArt.CuFunction(md, "vmuldiv_sp")
    else
      error("unrecognized precision: SP or DP")
    end
    # constants 1 and 0
    oneconst = one(eltype(d_X))
    zeroconst = zero(eltype(d_X))

    # pre-allocate variables on GPU
    d_Vnum = CUDArt.CudaArray(eltype(d_V), size(d_V))
    d_Vden = CUDArt.CudaArray(eltype(d_V), size(d_V))
    d_Wnum = CUDArt.CudaArray(eltype(d_W), size(d_W))
    d_Wden = CUDArt.CudaArray(eltype(d_W), size(d_W))
    d_B = CUDArt.CudaArray(eltype(d_X), size(d_X))
    d_XminusB = CUDArt.CudaArray(eltype(d_X), size(d_X))

    # initial objective value
    CUBLAS.gemm!('N', 'N', oneconst, d_V, d_W, zeroconst, d_B)
    CUBLAS.blascopy!(mn, d_X, 1, d_XminusB, 1)
    CUBLAS.axpy!(mn, -oneconst, d_B, 1, d_XminusB, 1)
    obj = CUBLAS.nrm2(mn, d_XminusB, 1); obj = obj * obj

    for iter = 1:maxiter

      # V = V .* (X * W') ./ (B * W')
      CUBLAS.gemm!('N', 'T', oneconst, d_X, d_W, zeroconst, d_Vnum)
      CUBLAS.gemm!('N', 'T', oneconst, d_B, d_W, zeroconst, d_Vden)
      CUDArt.launch(vmuldiv, m, r, (d_Vnum, d_Vden, d_V))
      # B = V * W
      CUBLAS.gemm!('N', 'N', oneconst, d_V, d_W, zeroconst, d_B)
      # W = W .* (V' * X) ./ (V' * B)
      CUBLAS.gemm!('T', 'N', oneconst, d_V, d_X, zeroconst, d_Wnum)
      CUBLAS.gemm!('T', 'N', oneconst, d_V, d_B, zeroconst, d_Wden)
      CUDArt.launch(vmuldiv, r, n, (d_Wnum, d_Wden, d_W))
      # B = V * W
      CUBLAS.gemm!('N', 'N', oneconst, d_V, d_W, zeroconst, d_B)

      # check convergence
      objold = obj
      CUBLAS.blascopy!(mn, d_X, 1, d_XminusB, 1)
      CUBLAS.axpy!(mn, -oneconst, d_B, 1, d_XminusB, 1)
      obj = CUBLAS.nrm2(mn, d_XminusB, 1); obj = obj * obj
      if abs(obj - objold) < tolfun * (abs(obj) + 1.0)
        break
      end

    end

    # transfer result to host and output
    V = CUDArt.to_host(d_V)
    W = CUDArt.to_host(d_W)
    return V, W

  elseif device == "cvx"

    for iter = 1:maxiter
      if mod(iter, 2) == 1
        # update V
        V = Convex.Variable(m, r)
        problem = Convex.minimize(vecnorm(X - V * W, 2))
        problem.constraints += V >= 0
      else
        # update W
        W = Convex.Variable(r, n)
        problem = Convex.minimize(vecnorm(X - V * W, 2))
        problem.constraints += W >= 0
      end
      Convex.solve!(problem)
    end

  else

    error("unrecognized device: CPU or GPU")

  end

end

nnmf (generic function with 1 method)

Debug and profile CPU code using r = 30.

In [5]:
r = 30
V0 = V0full[:, 1:r]
W0 = W0full[1:r, :]
gc()
#Profile.clear()
#profile V, W = nnmf(X, r; V=V0, W=W0)
#Profile.print(format=:flat)
@time V, W = nnmf(X, r; V=copy(V0), W=copy(W0));
println(vecnorm(X - V * W)^2)

  5.718640 seconds (937.55 k allocations: 59.388 MB, 0.17% gc time)
6621.627345486279


Debug and profile GPU code using r = 30. Before that make sure we use NVIDIA's `nvcc` compiler to compile our kernel function `vecop.cu`.

In [6]:
run(`cat vecop.cu`)
run(`nvcc -ptx -gencode=arch=compute_20,code=sm_20 vecop.cu`)

extern "C"
{

__global__ void vmuldiv_dp(const double *a, const double *b, double *c)
{
  int idx = threadIdx.x + blockIdx.x * blockDim.x;
  c[idx] *= a[idx] / b[idx];
}

__global__ void vmuldiv_sp(const float *a, const float *b, float *c)
{
  int idx = threadIdx.x + blockIdx.x * blockDim.x;
  c[idx] *= a[idx] / b[idx];
}

}


In [7]:
using CUDArt, CUBLAS
@time V, W = nnmf(X, r; V=copy(V0), W=copy(W0), device=:GPU, precision=:SP);
println(vecnorm(X - V * W)^2)
@time V, W = nnmf(X, r; V=copy(V0), W=copy(W0), device=:GPU, precision=:DP);
println(vecnorm(X - V * W)^2)

  4.730161 seconds (457.46 k allocations: 24.510 MB, 0.10% gc time)
6621.627018329714
  8.901734 seconds (204.69 k allocations: 9.660 MB)
6621.627345486279


Now let's time our algorithm at ranks r = 10, 20, 30, 40 and 50.

In [8]:
rlist = 10:10:50
runtime = zeros(length(rlist), 3)
objval = zeros(length(rlist), 3)
for i = 1:length(rlist)
    # rank
    r = rlist[i]
    println("rank = $r")
    # starting value
    V0 = V0full[:, 1:r]
    W0 = W0full[1:r, :]
    # CPU version
    runtime[i, 1] = @elapsed V, W = nnmf(X, r; V=copy(V0), W=copy(W0))
    objval[i, 1] = vecnorm(X - V * W)^2
    # GPU-SP version
    runtime[i, 2] = @elapsed V, W = nnmf(X, r; V=copy(V0), W=copy(W0), 
        device=:GPU, precision=:SP)
    objval[i, 2] = vecnorm(X - V * W)^2
    # GPU-DP version
    runtime[i, 3] = @elapsed V, W = nnmf(X, r; V=copy(V0), W=copy(W0), 
        device=:GPU, precision=:DP)
    objval[i, 3] = vecnorm(X - V * W)^2
end

rank = 10
rank = 20
rank = 30
rank = 40
rank = 50


Display timing results.

In [9]:
[runtime objval/1e3]

5x6 Array{Float64,2}:
  3.22638  1.30994   3.20499  11.7304   11.7304   11.7304 
  4.86586  2.85439   6.60161   8.49722   8.49722   8.49722
  7.169    4.40029   8.56611   6.62163   6.62163   6.62163
  8.39851  5.24118  14.741     5.25666   5.25666   5.25666
 11.7062   3.9505   21.4917    4.4302    4.4302    4.4302 